# 读取数据集

In [18]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('./data/train_set.csv')#训练集

test = pd.read_csv('./data/test_set.csv')#测试集

In [19]:
train.head()

,id,article,word_seg,class,label,class_trans
0,0,7368 1252069 365865 755561 1044285 129532 1053...,816903 597526 520477 1179558 1033823 758724 63...,14,13,14
1,1,581131 165432 7368 957317 1197553 570900 33659...,90540 816903 441039 816903 569138 816903 10343...,3,2,3
2,2,7368 87936 40494 490286 856005 641588 145611 1...,816903 1012629 957974 1033823 328210 947200 65...,12,11,12
3,3,299237 760651 299237 887082 159592 556634 7489...,563568 1239563 680125 780219 782805 1033823 19...,13,12,13
4,4,7368 7368 7368 865510 7368 396966 995243 37685...,816903 816903 816903 139132 816903 312320 1103...,12,11,12


# TFIDF构建文本特征

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [5]:
word_vec = TfidfVectorizer(analyzer='word',
            ngram_range=(1,2),#(1,3)
            min_df=3,  # 4  5
            max_df=0.9, # 0.95 1.0 
            use_idf=True,
            smooth_idf=True, 
            sublinear_tf=True)

In [6]:
train_term_doc = word_vec.fit_transform(train['word_seg'])

In [7]:
test_term_doc = word_vec.transform(test['word_seg'])

In [20]:
train_term_doc#sparse matrix 102277x2820641  np.array

<102277x2820641 sparse matrix of type '<class 'numpy.float64'>'
	with 75739932 stored elements in Compressed Sparse Row format>

In [21]:
test_term_doc  #102277x2820641 稀疏矩阵可以直接在逻辑回归里面调用

<102277x2820641 sparse matrix of type '<class 'numpy.float64'>'
	with 73582169 stored elements in Compressed Sparse Row format>

# 逻辑回归构建模型

In [ ]:
[1,2,3,4,5,6,7...,19]  -> [0,1,2,...18]
["a",'b',...]

In [22]:
from sklearn.preprocessing import LabelEncoder

In [23]:
lb = LabelEncoder()
train['label'] = lb.fit_transform(train['class'])

In [24]:
lb.fit_transform(train['class'])

array([13,  2, 11, ..., 11,  3, 10])

In [25]:
train.head()

,id,article,word_seg,class,label,class_trans
0,0,7368 1252069 365865 755561 1044285 129532 1053...,816903 597526 520477 1179558 1033823 758724 63...,14,13,14
1,1,581131 165432 7368 957317 1197553 570900 33659...,90540 816903 441039 816903 569138 816903 10343...,3,2,3
2,2,7368 87936 40494 490286 856005 641588 145611 1...,816903 1012629 957974 1033823 328210 947200 65...,12,11,12
3,3,299237 760651 299237 887082 159592 556634 7489...,563568 1239563 680125 780219 782805 1033823 19...,13,12,13
4,4,7368 7368 7368 865510 7368 396966 995243 37685...,816903 816903 816903 139132 816903 312320 1103...,12,11,12


In [26]:
train['class_trans'] = lb.inverse_transform(train['label'])

In [27]:
train.head()

,id,article,word_seg,class,label,class_trans
0,0,7368 1252069 365865 755561 1044285 129532 1053...,816903 597526 520477 1179558 1033823 758724 63...,14,13,14
1,1,581131 165432 7368 957317 1197553 570900 33659...,90540 816903 441039 816903 569138 816903 10343...,3,2,3
2,2,7368 87936 40494 490286 856005 641588 145611 1...,816903 1012629 957974 1033823 328210 947200 65...,12,11,12
3,3,299237 760651 299237 887082 159592 556634 7489...,563568 1239563 680125 780219 782805 1033823 19...,13,12,13
4,4,7368 7368 7368 865510 7368 396966 995243 37685...,816903 816903 816903 139132 816903 312320 1103...,12,11,12


In [29]:
train[train['class'] == train['class_trans']].shape

(102277, 6)

In [14]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(C=4, dual=True) # c 1.0  2 3 4 5  200w 10w
clf.fit(train_term_doc,train['label'])#[0,18]


/home/hcq/miniconda3/envs/python3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/hcq/miniconda3/envs/python3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=4, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [30]:
test_prob = clf.predict_proba(test_term_doc)

In [33]:
test_prob[0]# 4 +1 =5

array([0.00959619, 0.00508457, 0.00357474, 0.01217717, 0.86841463,
       0.00547256, 0.00854222, 0.0035889 , 0.00909855, 0.00317351,
       0.0074878 , 0.00535788, 0.0062232 , 0.02057748, 0.01153442,
       0.00162819, 0.00505501, 0.00347785, 0.00993513])

# 结果提交

In [34]:
test_pred = np.argmax(test_prob,axis=1)

In [35]:
test_pred

array([ 4,  3, 12, ...,  5, 13, 12])

In [36]:
test['class'] = lb.inverse_transform(test_pred)#[1,...19]

In [37]:
test.head()

,id,article,word_seg,class
0,0,7368 146447 316564 42610 55736 297797 93042 53...,816903 565958 726082 764656 335008 75094 20282...,5
1,1,985531 473628 1044285 1121849 206763 462208 11...,729468 520477 529032 101368 335130 520477 1113...,4
2,2,7368 7368 7368 7368 7368 7368 7368 7368 7368 7...,816903 816903 816903 816903 816903 816903 8169...,13
3,3,529819 1226459 856005 1177293 663773 272235 93...,231664 1033823 524850 330478 507199 520477 618...,4
4,4,42610 1252069 1077049 955883 1125260 1044285 2...,545370 379223 162767 520477 1194630 1197475 11...,5


In [38]:
test[["id","class"]].to_csv("submission_baseline.csv",index=False,header=True,encoding='utf-8')